## Setup

In [432]:
from __future__ import annotations
import typing
import urllib.request
import urllib.response
import urllib.parse
import http.client
import json
import pathlib
import subprocess
import collections
import time

In [433]:
HERE = pathlib.Path(".").absolute()

In [434]:
class ServiceError(Exception):
    def __init__(self, code: http.HTTPStatus, message: str) -> None:
        super().__init__(f"Error {code.value} {code.phrase}: {message}")

In [435]:
class NonRaisingHTTPErrorProcessor(urllib.request.HTTPErrorProcessor):
    http_response = https_response = lambda self, request, response: response

opener = urllib.request.build_opener(NonRaisingHTTPErrorProcessor)

In [436]:
class Microservice:
    path: pathlib.Path
    port: int
    port_private: int | None
    process: subprocess.Popen
    
    def __init__(self, path: pathlib.Path, port: int, port_private: int | None = None) -> None:
        self.path = path
        self.port = port
        self.port_private = port_private
    
    def start(self) -> None:
        self.process = subprocess.Popen(
            ["go", "run", "main.go"],
            cwd=self.path,
        )
    
    def stop(self) -> None:
        self.process.terminate()
        self.process.wait()
        self.process.kill()
    
    def request(
        self,
        uri: str,
        data: bytes | str | None = None,
        headers: dict[str, str] = {},
        method: str = "GET",
        private: bool = False,
    ) -> str:
        port: int = self.port_private if private else self.port
        assert port is not None
        # print(f"!!! requesting {method} http://localhost:{port}{uri}")
        request = urllib.request.Request(
            url=f"http://localhost:{port}{uri}",
            data=data,
            headers=headers,
            method=method,
        )
        response: http.client.HTTPResponse = opener.open(request)
        body: str = response.read().decode().strip()
        if not (200 <= response.status < 300):
            raise ServiceError(http.HTTPStatus(response.status), body)
        return body


In [437]:
class UserSvc(Microservice):
    def __init__(self):
        super().__init__(
            HERE.parent / "user-service",
            8084,
            8083,
        )
    
    def _request(
        self,
        uri: str,
        data: typing.Any,
        method: str = "GET",
        private: bool = False,
    ) -> typing.Any:
        resp = self.request(
            uri=uri,
            data=json.dumps(data).encode(),
            method=method,
            private=private,
        ).strip()
        
        if not resp:
            return None
        return json.loads(resp)
    
    def login(self, login: str, password: str) -> str:
        return self._request(
            "/user/login",
            {
                "login": login,
                "password": password,
            },
            method="POST",
        )
    
    def create(self, login: str, password: str) -> dict[typing.Literal["id"], int]:
        return self._request(
            "/user/create",
            {
                "login": login,
                "password": password,
            },
            method="POST",
        )
    
    def delete(self, token: str) -> None:
        return self._request(
            "/user/delete",
            {
                "token": token,
            },
            method="POST",
        )
    
    def edit(self, token: str, id: str, new_login: str, new_password: str) -> None:
        return self._request(
            "/user/edit",
            {
                "token": token,
                "id": id,
                "newLogin": new_login,
                "newPassword": new_password,
            },
            method="POST",
        )
    
    def set_permissions(self, token: str, id: str, permissions: int) -> None:
        return self._request(
            "/user/edit",
            {
                "token": token,
                "id": id,
                "permission": permissions,
            },
            method="POST",
        )
    
    def refresh_token(self, token: str, refresh: str) -> str:
        return self._request(
            "/user/refresh",
            {
                "token": token,
                "refresh": refresh,
            },
            method="POST",
        )
    
    def private_get_id(self, token: str) -> dict[typing.Literal["id"], int]:
        return self._request(
            "/user/id",
            {
                "token": token,
            },
            method="POST",
            private=True,
        )
    
    # The typo is deliberate, as it is in the actual service
    def private_get_permissions(self, token: str) -> dict[typing.Literal["permissios"], int]:
        return self._request(
            "/user/permissions",
            {
                "token": token,
            },
            method="POST",
            private=True,
        )


In [438]:
class _ReadRecorder(collections.UserDict):
    accessed_keys: set[typing.Any]
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.accessed_keys = set()
    
    def __getitem__(self, key: str) -> typing.Any:
        result = super().__getitem__(key)
        self.accessed_keys.add(key)
        return result

In [439]:
class BookSvc(Microservice):
    def __init__(self):
        super().__init__(
            HERE.parent / "book-service",
            8082,
        )
    
    def _request(
        self,
        uri: str,
        query: dict[str, str] = {},
        json_body: typing.Any | None = None,
        auth: str | None = None,
        method: str = "GET",
    ) -> typing.Any:
        headers = {}
        if auth is not None:
            headers["Authorization"] = auth
        
        tmp_query = _ReadRecorder({
            key: urllib.parse.quote(value) for key, value in query.items()
        })
        uri = uri.format_map(tmp_query)
        query = {key: value for key, value in query.items() if key not in tmp_query.accessed_keys}
        del tmp_query
        
        query = urllib.parse.urlencode(query)
        if method == "GET":
            uri += f"?{query}"
            query = None
        
        resp = self.request(
            uri=uri,
            data=json.dumps(json_body).encode() or query if method != "GET" else None,
            headers=headers,
            method=method,
        ).strip()
        
        if not resp:
            return None
        return json.loads(resp)

    def get_books(self, criteria: str | None = None) -> list[dict[str, typing.Any]]:
        return self._request(
            "/api/v1/books",
            query={
                "criteria": criteria or "",
            },
            method="GET",
        )
    
    def get_book(self, id: str) -> dict[str, typing.Any]:
        return self._request(
            "/api/v1/books/{id}",
            query={
                "id": id,
            },
            method="GET",
        )
    
    def create_book(self, token: str, data: dict[str, typing.Any]) -> dict[str, typing.Any]:
        return self._request(
            "/api/v1/books/new",
            json_body=data,
            auth=token,
            method="POST",
        )
    
    def update_book(self, token: str, id: str, data: dict[str, typing.Any]) -> dict[str, typing.Any]:
        return self._request(
            "/api/v1/books/{id}",
            query={
                "id": id,
            },
            json_body=data,
            auth=token,
            method="POST",
        )
    
    def delete_book(self, token: str, id: str) -> None:
        return self._request(
            "/api/v1/books/{id}",
            query={
                "id": id,
            },
            auth=token,
            method="DELETE",
        )


In [440]:
class LoanSvc(Microservice):
    def __init__(self):
        super().__init__(
            HERE,
            8080,
            8081,
        )
    
    def _request(
        self,
        uri: str,
        query: dict[str, str] = {},
        method: str = "GET",
        private: bool = False,
    ) -> typing.Any:
        tmp_query = _ReadRecorder(query)
        uri = uri.format_map(tmp_query)
        query = {key: value for key, value in query.items() if key not in tmp_query.accessed_keys}
        del tmp_query
        
        query = urllib.parse.urlencode(query)
        if method == "GET":
            uri += f"?{query}"
            query = None
        else:
            query = query.encode()
        
        resp = self.request(
            uri=uri,
            data=query if method != "GET" else None,
            method=method,
            private=private,
        ).strip()
        
        if not resp:
            return None
        return json.loads(resp)
    
    def take_book(self, token: str, book_id: str, user_id: str | None = None) -> dict[None, None]:
        return self._request(
            "/api/v1/book/{bookID}/take",
            query={
                "auth": token,
                "bookID": book_id,
                "user": user_id or "",
            },
            method="POST",
        )
    
    def return_book(self, token: str, book_id: str, user_id: str | None = None) -> dict[None, None]:
        return self._request(
            "/api/v1/book/{bookID}/return",
            query={
                "auth": token,
                "bookID": book_id,
                "user": user_id or "",
            },
            method="POST",
        )
    
    def count_available(self, token: str, book_id: str) -> dict[typing.Literal["available"], int]:
        return self._request(
            "/api/v1/book/{bookID}/avail",
            query={
                "auth": token,
                "bookID": book_id,
            },
            method="GET",
        )
    
    def get_reserved(self, token: str, at_time: float | None = None) -> dict[typing.Literal["reserved"], list[dict[str, typing.Any]]]:
        if at_time is None:
            at_time = ""
        else:
            at_time = int(at_time)
        return self._request(
            "/api/v1/reserved",
            query={
                "auth": token,
                "atTime": at_time,
            },
            method="GET",
        )
    
    def get_overdue(self, token: str, at_time: float | None = None) -> dict[typing.Literal["overdue"], list[dict[str, typing.Any]]]:
        if at_time is None:
            at_time = ""
        else:
            at_time = int(at_time)
        return self._request(
            "/api/v1/overdue",
            query={
                "auth": token,
                "atTime": at_time,
            },
            method="GET",
        )
    
    def get_user_loans(self, user_id: str) -> dict[typing.Literal["unreturned"], int]:
        return self._request(
            "/api/v1/userloans/{userID}",
            query={
                "userID": user_id,
            },
            method="GET",
            private=True,
        )


In [441]:
users = UserSvc()
books = BookSvc()
loans = LoanSvc()

In [442]:
def start_all() -> None:
    users.start()
    books.start()
    loans.start()

# TODO: Doesn't work --- needs recursive killing
# def stop_all() -> None:
#     loans.stop()
#     books.stop()
#     users.stop()

## Demo

In [443]:
start_all()

### Users

In [161]:
token = users.login("admin", "superuser")
token

{'Access': '23c7fdf1a3002f3c0455b7f3212a33a7fd11ff680da152cb1794b0ca1aaa3b3f9cac1dbc0e78834266e67d74ae6ff70051ff545fbbf2fbc6fa573bb07b366555',
 'Refresh': '6a2cd0485229981b3ad5b38087daa7d611e55bcf452fa5c786f6a3a25c933dfa7e85d011fdef23d307d1dc6fdc0f5d861e7cce7df6f3a311eb47f8d20f95d35f',
 'Expiration': '2024-12-11T21:18:52.3299181+03:00'}

In [162]:
auth = token["Access"]

In [163]:
users.create("pupa", "lupa")

{'id': '2'}

In [164]:
users.login("pupa", "lupa")

{'Access': '0a381cc7486d3b19188e3c48d58158856d6a0250b8dbc2f33fa73e251ae051d612a657472fd1f9b101f5d58d34bd608629c6d32de289aac7fb08c01f555fe732',
 'Refresh': 'b158d67b42f13b49c30149fa0bbe38a817bbe50000b30c9a88928f8adbccc686802204a533e44b94790fca6ad0e0ccb052c16f9b29364303bcfe929c85cb66c8',
 'Expiration': '2024-12-11T21:19:01.1462115+03:00'}

In [165]:
users.edit(auth, "2", "pupa", "nelupa")

In [ ]:
# users.login("pupa", "lupa")

ServiceError: Error 400 Bad Request: Error getting token

In [168]:
users.login("pupa", "nelupa")

{'Access': '186cbfc5bed49333f46b9dc779bdd904ce26afadd4ee1bb4ee82775c948b40224bb27721f48578db4a82708bc8b78771554a928ae830aa2c0d8fb1671bf9162f',
 'Refresh': 'ebdb453df0d86e74578420aa6f106fd9ae3d831318b03a6d8468d616d559c826b673ac08541ad9c4f765867afa97a014acba95b82db106f68f324e7be599b7fd',
 'Expiration': '2024-12-11T21:19:25.6446216+03:00'}

In [303]:
users.private_get_id(auth)

{'id': '1'}

In [304]:
users.private_get_permissions(auth)

{'permissios': '17895697'}

### Books

In [239]:
auth = users.login("admin", "superuser")["Access"]

In [222]:
books.get_books()

[{'id': '1',
  'title': 'Go Programming Language',
  'author': 'Alan Donovan, Kernigan',
  'description': 'Good one',
  'stock': '101'}]

In [223]:
books.get_books("Bible")

[]

In [224]:
books.get_books("Kernigan")

[{'id': '1',
  'title': 'Go Programming Language',
  'author': 'Alan Donovan, Kernigan',
  'description': 'Good one',
  'stock': '101'}]

In [225]:
books.get_book("1")

{'id': '1',
 'title': 'Go Programming Language',
 'author': 'Alan Donovan, Kernigan',
 'description': 'Good one',
 'stock': '101'}

In [ ]:
# books.get_book("0")

ServiceError: Error 500 Internal Server Error: Failed to get book by ID: Could not load books: Book not found

In [242]:
books.create_book(
    auth,
    {
        "id": "2",
        "title": "The Bible",
        "author": "God Almightly",
        "description": "Also fine",
        "stock": "0",
    },
)

{'id': '2'}

In [243]:
books.get_books()

[{'id': '1',
  'title': 'Go Programming Language',
  'author': 'Alan Donovan, Kernigan',
  'description': 'Good one',
  'stock': '101'},
 {'id': '2',
  'title': 'The Bible',
  'author': 'God Almightly',
  'description': 'Also fine',
  'stock': '0'}]

In [244]:
books.delete_book(auth, "2")

### Loans

In [444]:
auth = users.login("admin", "superuser")["Access"]

In [456]:
loans.count_available(auth, "1")

{'available': 101}

In [457]:
loans.take_book(auth, "1")

{}

In [458]:
time_taken = time.time()

In [459]:
loans.count_available(auth, "1")

{'available': 100}

In [460]:
loans.get_user_loans("1")

{'unreturned': 1}

In [461]:
loans.get_reserved(auth)

{'reserved': [{'id': 'b9ace95e-26d8-4f41-b206-e6f11f9c63c6',
   'user_id': '1',
   'book_id': '1',
   'taken_at': 1733943944,
   'return_deadline': 1735153544,
   'returned': False,
   'returned_at': 0}]}

In [462]:
loans.get_overdue(auth, at_time=time.time() + 60*60)

{'overdue': []}

In [463]:
loans.get_overdue(auth, at_time=time.time() + 15*24*60*60)

{'overdue': [{'id': 'b9ace95e-26d8-4f41-b206-e6f11f9c63c6',
   'user_id': '1',
   'book_id': '1',
   'taken_at': 1733943944,
   'return_deadline': 1735153544,
   'returned': False,
   'returned_at': 0}]}

In [464]:
loans.return_book(auth, "1")

{}

In [465]:
loans.get_reserved(auth)

{'reserved': []}

In [466]:
loans.get_reserved(auth, at_time=time_taken)

{'reserved': [{'id': 'b9ace95e-26d8-4f41-b206-e6f11f9c63c6',
   'user_id': '1',
   'book_id': '1',
   'taken_at': 1733943944,
   'return_deadline': 1735153544,
   'returned': True,
   'returned_at': 1733943952}]}

In [467]:
loans.get_user_loans("1")

{'unreturned': 0}